In [11]:
from astropy.io import fits
from astropy.stats import sigma_clip
import numpy as np

In [12]:

def create_median_dark(dark_list, bias_filename, median_dark_filename):
    """This function:

    - Accept a list of dark file paths to combine as dark_list.
    - Accept a median bias frame filename as bias_filename (the one you created using
      create_median_bias).
    - Read all the images in dark_list and create a list of 2D numpy arrays.
    - Read the bias frame.
    - Subtract the bias frame from each dark image.
    - Divide each dark image by its exposure time so that you get the dark current
      per second. The exposure time can be found in the header of the FITS file.
    - Use a sigma clipping algorithm to combine all the bias-corrected dark frames
      using the median and removing outliers outside 3-sigma for each pixel.
    - Save the resulting dark frame to a FITS file with the name median_dark_filename.
    - Return the median dark frame as a 2D numpy array.

    """
    #Step Get data ()
    bias_data = fits.getdata(bias_filename).astype('f4')
    dark_frames = []
    #Step getting exposure time for each file and normalizing it. Also put them in float 32 arrays
    for filename in dark_list:
            dark = fits.open(filename)
            data = dark[0].data.astype('f4')
            exptime = dark[0].header['EXPTIME']
            dark_corrected = (data - bias_data) / exptime
            dark_frames.append(dark_corrected)
    dark_stack = np.array(dark_frames)
    #Step sigma clipping
    clipped = sigma_clip(dark_stack, sigma=3.0, axis=0, cenfunc='median')
    #Step average out
    median_dark = np.ma.mean(clipped, axis=0).data
    #Step Saving
    hdu = fits.PrimaryHDU(data=median_dark)
    hdu.writeto(median_dark_filename, overwrite=True)
    #Step returning the result when called out
    return median_dark


In [13]:
# Your test list of dark frames
dark_files = [
    '/home/jovyan/work/ccd-reductions-Jbhandol/data/ccd_reductions_data/Dark-S001-R001-C001-NoFilt.fit',
    '/home/jovyan/work/ccd-reductions-Jbhandol/data/ccd_reductions_data/Dark-S001-R001-C002-NoFilt.fit',
    '/home/jovyan/work/ccd-reductions-Jbhandol/data/ccd_reductions_data/Dark-S001-R001-C003-NoFilt.fit',
    '/home/jovyan/work/ccd-reductions-Jbhandol/data/ccd_reductions_data/Dark-S001-R001-C004-NoFilt.fit',
    '/home/jovyan/work/ccd-reductions-Jbhandol/data/ccd_reductions_data/Dark-S001-R001-C005-NoFilt.fit',
    '/home/jovyan/work/ccd-reductions-Jbhandol/data/ccd_reductions_data/Dark-S001-R001-C006-NoFilt.fit'
]

median_dark = create_median_dark(
    dark_list=dark_files,
    bias_filename='bias_median.fits',
    median_dark_filename='dark_median.fits'
)


In [14]:
from astropy.io import fits
import numpy as np

# Individual darks (use same dark_files list)
print("📂 Individual Dark Frame Stats:")
for file in dark_files:
    data = fits.getdata(file).astype('f4')
    print(f"{file.split('/')[-1]} - Mean: {np.mean(data):.2f}, Std: {np.std(data):.2f}")

# Combined dark frame
dark_median = fits.getdata('dark_median.fits')
print("\n📦 Combined Median Dark Frame:")
print(f"Mean: {np.mean(dark_median):.4f}")
print(f"Std : {np.std(dark_median):.4f}")


📂 Individual Dark Frame Stats:
Dark-S001-R001-C001-NoFilt.fit - Mean: 1062.03, Std: 43.41
Dark-S001-R001-C002-NoFilt.fit - Mean: 1058.77, Std: 43.33
Dark-S001-R001-C003-NoFilt.fit - Mean: 1056.83, Std: 43.33
Dark-S001-R001-C004-NoFilt.fit - Mean: 1057.65, Std: 43.30
Dark-S001-R001-C005-NoFilt.fit - Mean: 1057.39, Std: 43.35
Dark-S001-R001-C006-NoFilt.fit - Mean: 1057.15, Std: 43.36

📦 Combined Median Dark Frame:
Mean: 0.0652
Std : 0.4704
